In [19]:
import pandas as pd
import quandl # dataset providing package, now iam using kaggle for downloading sample data
import math

In [10]:
df = pd.read_csv('data_files/WIKI_PRICES.csv')
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,31.041951,34.112034,27.289627,30.018590,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,29.295415,29.336350,27.160002,27.548879,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,28.183363,30.018590,27.330562,30.018590,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,28.995229,29.766161,27.460188,27.460188,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,27.378319,28.613174,27.289627,28.012803,3464400.0


In [11]:
df = df[['adj_open','adj_high','adj_low','adj_close','adj_volume']]

In [12]:
# high low percentage
df['HL_PCT'] = (df['adj_high'] - df['adj_close']) / df['adj_close'] * 100.0

In [13]:
# percentage change new - old / old * 100.0
df['PCT_change'] = (df['adj_close'] - df['adj_open']) / df['adj_open'] * 100.0

In [15]:
df = df[['adj_close','HL_PCT','PCT_change','adj_volume']]

In [16]:
df.head()

,adj_close,HL_PCT,PCT_change,adj_volume
0,30.018590,13.636364,-3.296703,44739900.0
1,27.548879,6.488361,-5.961807,10897100.0
2,30.018590,0.000000,6.511740,4705200.0
3,27.460188,8.397516,-5.294118,4274400.0
4,28.012803,2.143205,2.317468,3464400.0


In [17]:
forecast_col = 'adj_close'

In [18]:
df.fillna(-99999, inplace=True)

/tmp/ipykernel_25428/748744614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(-99999, inplace=True)


In [21]:
forcast_out = int(math.ceil(0.1 * len(df))) # 10 percentage data

In [22]:
df['label'] = df[forecast_col].shift(-forcast_out)

/tmp/ipykernel_25428/1782054120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df[forecast_col].shift(-forcast_out)


In [23]:
df.head()

,adj_close,HL_PCT,PCT_change,adj_volume,label
0,30.018590,13.636364,-3.296703,44739900.0,27.130393
1,27.548879,6.488361,-5.961807,10897100.0,27.400187
2,30.018590,0.000000,6.511740,4705200.0,26.956626
3,27.460188,8.397516,-5.294118,4274400.0,23.380701
4,28.012803,2.143205,2.317468,3464400.0,22.553025


In [ ]:
df.tail()